# Kaggle Titanic

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

# 簡易EDA

In [2]:
# データの読み込み
# Submission目的なのでカラムは扱いやすいものだけ抜き出す
train = pd.read_csv('input/train.csv', usecols=['Survived', 'Pclass', 'Embarked', 'Fare'])
test = pd.read_csv('input/test.csv', usecols=['Pclass', 'Embarked', 'Fare'])

train.columns, test.columns

(Index(['Survived', 'Pclass', 'Fare', 'Embarked'], dtype='object'),
 Index(['Pclass', 'Fare', 'Embarked'], dtype='object'))

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 27.9+ KB


In [4]:
# 欠損の数
train.isnull().sum()

Survived    0
Pclass      0
Fare        0
Embarked    2
dtype: int64

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 3 columns):
Pclass      418 non-null int64
Fare        417 non-null float64
Embarked    418 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 9.9+ KB


In [6]:
test.isnull().sum()

Pclass      0
Fare        1
Embarked    0
dtype: int64

In [7]:
train['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [8]:
test['Embarked'].unique()

array(['Q', 'S', 'C'], dtype=object)

In [9]:
train.describe()

,Survived,Pclass,Fare
count,891.000000,891.000000,891.000000
mean,0.383838,2.308642,32.204208
std,0.486592,0.836071,49.693429
min,0.000000,1.000000,0.000000
25%,0.000000,2.000000,7.910400
50%,0.000000,3.000000,14.454200
75%,1.000000,3.000000,31.000000
max,1.000000,3.000000,512.329200


In [10]:
test.describe()

,Pclass,Fare
count,418.000000,417.000000
mean,2.265550,35.627188
std,0.841838,55.907576
min,1.000000,0.000000
25%,1.000000,7.895800
50%,3.000000,14.454200
75%,3.000000,31.500000
max,3.000000,512.329200


## データの前処理
- とにかくLogisticRegressionに突っ込めればいい
    - 欠損は適当に埋める
    - 文字データは数値データに置換する

In [11]:
# Embarkedの欠損を埋める＆文字データを数値データに置換する

# 最頻値である S で欠損を補完する
train['Embarked'] = train['Embarked'].fillna('S')

train['Embarked'] = train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
train['Embarked'].value_counts()

0    646
1    168
2     77
Name: Embarked, dtype: int64

In [12]:
train.head()

,Survived,Pclass,Fare,Embarked
0,0,3,7.2500,0
1,1,1,71.2833,1
2,1,3,7.9250,0
3,1,1,53.1000,0
4,0,3,8.0500,0


# データの分割

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns=['Survived']),
                                                    train['Survived'],
                                                    random_state=0)

X_train.shape, X_valid.shape

((668, 3), (223, 3))

# LogisticRegressionで訓練

In [14]:
from sklearn.linear_model import LogisticRegression

logreg1 = LogisticRegression(C=1)
logreg1.fit(X_train, y_train)

logreg1.score(X_valid, y_valid)

0.7085201793721974

In [15]:
logreg2 = LogisticRegression(C=10)
logreg2.fit(X_train, y_train)

logreg2.score(X_valid, y_valid)

0.7040358744394619

In [22]:
logreg3 = LogisticRegression(C=100)
logreg3.fit(X_train, y_train)

logreg3.score(X_valid, y_valid)

0.7040358744394619

## 良かったモデルで再学習

In [16]:
final_model = LogisticRegression(C=1)
final_model.fit(train.drop(columns=['Survived']), train['Survived'])



LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## テストデータでの予測
- 訓練データと同じ前処理をすること

In [17]:
test['Embarked'] = test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

test['Fare'] = test['Fare'].fillna(train['Fare'].median())

In [18]:
test.head()

,Pclass,Fare,Embarked
0,3,7.8292,2
1,3,7.0000,0
2,2,9.6875,2
3,3,8.6625,0
4,3,12.2875,0


In [19]:
final_model.predict(test)

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [20]:
submit_data = pd.read_csv('input/gender_submission.csv')

submit_data['Survived'] = final_model.predict(test)

# CSVで保存
# submit_data.to_csv('20191004_01_logreg1.csv', index=False)